# Second Round of Genotype Modeling

This notebook follows 'ModelGenotypes.ipynb' to be a notebook for a second round of genotype modeling using alternative strategies to dimensionality reduce and model genotypes to phenotypes

In [11]:
import sgkit as sg
from sgkit.io import plink
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import MultiTaskElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from joblib import Parallel, delayed
import xgboost as xg
import pandas as pd
import numpy as np
import math
import dask
import pickle
import xarray as xr
import umap.umap_ as UMAP

## MUST RUN: Get Genotype Data To a Matrix

In [2]:
ds = plink.read_plink(bed_path = 'ratgenes_pruned.bed', bim_path = 'ratgenes_pruned.bim', fam_path = 'ratgenes_pruned.fam')
ds = ds.set_index({"samples": "sample_id"})
ds = ds.set_index({"variants": "variant_id"})
call_g_mask = ds["call_genotype_mask"].any(dim = "ploidy")
call_g = xr.where(call_g_mask, -1, ds["call_genotype"].sum(dim = "ploidy"))
genotypes_matrix = call_g.values
genotypes_matrix = np.transpose(genotypes_matrix)
genotypes_matrix.shape

(13526, 96654)

In [3]:
print(genotypes_matrix[:5])

[[ 2  0  0 ...  2  2  2]
 [ 0  2  2 ...  1  1  1]
 [-1  1  1 ...  2  2  2]
 [ 1  1  1 ...  0  0  0]
 [-1  0  0 ...  2  2  2]]


## MUST RUN: Load in loco phenotypes dataset

In [4]:
loco = pd.read_csv("pheno_loco_clean.txt", sep = '\t')

## MUST RUN: Code for cleaning other phenotype data in 'phenotype_chunks'

In [5]:
def clean_chunks(data):
    
    old = data
    
    new = data.set_index('rfid')
    new = new.select_dtypes(['number'])
    limit = math.floor(len(new) * 0.80)
    new = new.dropna(axis=1, thresh=limit)
    new = new.dropna(axis=0)
    
    return(new)

## MUST RUN: Code for dimensionality reducing

In [5]:
def pca(mat, n_components):
    
    mat_reduced = PCA(n_components = n_components, svd_solver = 'randomized').fit_transform(mat)
    return(mat_reduced)

def kernel_pca(mat, n_components):
    
    mat_reduced = KernelPCA(n_components = n_components, kernel = 'rbf', eigen_solver = 'randomized').fit_transform(mat)
    return(mat_reduced)

def umap(mat, metric, n_components):
    
    mat_reduced = UMAP.UMAP(metric = metric, n_neighbors = 15, min_dist = 0.1, n_components = n_components).fit_transform(mat)
    return(mat_reduced)

## MUST RUN: Create matrix filtered by select 'loco' phenotypes

In [6]:
def get_rats_with_loco(genotypes_mat, phenotypes):

    # Select only 'max' loco phenotypes
    loco_select = ['rfid', 'loco_maxcent', 'loco_maxdis', 'loco_maxrear', 'loco_maxact']
    pheno_select = phenotypes[loco_select].set_index('rfid')
    rat_ids = ds["samples"].values
    genotypes_df = pd.DataFrame(data = genotypes_mat, index = rat_ids)
    genotypes_df.index.name = 'rfid'
    geno_with_pheno = pd.merge(genotypes_df, pheno_select, left_index=True, right_index=True)

    # Split the geno_with_pheno into X and y datasets
    phenos = ['loco_maxcent', 'loco_maxdis', 'loco_maxrear', 'loco_maxact']
    y_pheno = geno_with_pheno[phenos]
    y_pheno = y_pheno.to_numpy()
    X_geno = geno_with_pheno.drop(columns = phenos)
    X_geno = X_geno.to_numpy()
    
    return(X_geno, y_pheno)

## MUST RUN: Create matrix with all phenotypes mapped to rat

In [13]:
def get_rats_with_pheno(genotypes_mat, phenotypes):

    rat_ids = ds["samples"].values
    genotypes_df = pd.DataFrame(data = genotypes_mat, index = rat_ids)
    genotypes_df.index.name = 'rfid'
    geno_with_pheno = pd.merge(genotypes_df, phenotypes, left_index=True, right_index=True)
    y_pheno = geno_with_pheno[phenotypes.columns].to_numpy()
    X_geno = geno_with_pheno.drop(columns = phenotypes.columns).to_numpy()
    
    return(X_geno, y_pheno)

# (A) Explore Various Modeling

## Option 1: Train ElasticNet Model to Predict loco phenotypes

In [9]:
X_geno, y_pheno = get_rats_with_pheno(genotypes_matrix, loco)

In [10]:
print(X_geno[:5])
print(y_pheno[:5])

[[-1 -1 -1 ...  0  0  0]
 [ 0  2  2 ...  0  0  0]
 [ 0  2  2 ...  2  2  2]
 [ 1  1  1 ...  0  0  0]
 [ 0  2  2 ...  2  2  2]]
[[171.3 568.   19.  109. ]
 [115.4 697.   26.  135. ]
 [ 48.1 652.   20.  124. ]
 [147.1 466.   25.   97. ]
 [ 89.4 599.   22.   96. ]]


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_geno, y_pheno, test_size=0.20, random_state=42)

In [10]:
regr = ElasticNet(max_iter = 10000, random_state = 42).fit(X_train, y_train)

In [13]:
y_train_pred = regr.predict(X_train)
print(y_train_pred[:5])

y_test_pred = regr.predict(X_test)
print(y_test_pred[:5])

[[ 68.91334236 630.49018608  31.20997093  98.80398388]
 [ 65.59016069 726.37038284  31.68254412  92.42864566]
 [ 44.08774537 745.75065656  31.43889058  91.96772778]
 [119.9160312  617.96965894  31.59672354  94.65726659]
 [ 51.30986502 730.26400249  32.15657582  86.78766635]]
[[ 87.23772221 700.29537406  31.81182426  88.40180429]
 [ 78.99918893 621.85176151  31.25046867  88.45242396]
 [ 70.27893374 624.14015041  31.50553736  86.76195827]
 [ 66.11747644 570.26462493  31.11727521  91.65692861]
 [ 71.00788591 541.16771272  31.474246    91.22443367]]


In [14]:
print("MAE Train:", mean_absolute_error(y_train, y_train_pred, multioutput='raw_values'))
print("MAE Test:", mean_absolute_error(y_test, y_test_pred, multioutput='raw_values'))
print("MSE Train:", mean_squared_error(y_train, y_train_pred, multioutput='raw_values'))
print("MSE Test:", mean_squared_error(y_test, y_test_pred, multioutput='raw_values'))
print("R2 Train:", r2_score(y_train, y_train_pred, multioutput = 'raw_values'))
print("R2 Test:", r2_score(y_test, y_test_pred, multioutput = 'raw_values'))

MAE Train: [18.34300385 28.91180835  5.64786636 11.50847375]
MAE Test: [34.21429027 85.35406017  5.42908778 14.28944104]
MSE Train: [ 549.79810229 1361.81790317   51.15431517  213.69389581]
MSE Test: [ 1845.9485962  11268.93340119    45.69795862   321.08439423]
R2 Train: [0.64437101 0.87323639 0.02725899 0.41403722]
R2 Test: [-0.02624217  0.01973684  0.00360182  0.05933711]


## Option 2: Train an Iteratively Fitted ElasticNet Model

In [12]:
X_geno, y_pheno = get_rats_with_pheno(genotypes_matrix, loco)

In [22]:
regr = MultiTaskElasticNetCV(max_iter = 10000, random_state = 42, n_jobs = -1).fit(X_geno, y_pheno)

In [23]:
r2_elastic_net = regr.score(X_geno, y_pheno)
print(r2_elastic_net)

0.26892481716522676


In [ ]:
import pickle

with open('elastic_net_cv.pkl', 'wb') as handle:
    pickle.dump(regr, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Option 3: Train a Random Forest on PCA-UMAP Reduced Data

In [15]:
geno_pca = pca(genotypes_matrix, 1000)
geno_pca_umap = umap(geno_pca, 'euclidean', 50)

In [10]:
X_geno, y_pheno = get_rats_with_pheno(geno_pca_umap, loco)

NameError: name 'geno_pca_umap' is not defined

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_geno, y_pheno, test_size=0.20)

In [19]:
regr = RandomForestRegressor(n_jobs = -1).fit(X_train, y_train)

In [20]:
y_train_pred = regr.predict(X_train)
y_test_pred = regr.predict(X_test)

In [21]:
print("MAE Train:", mean_absolute_error(y_train, y_train_pred, multioutput='raw_values'))
print("MAE Test:", mean_absolute_error(y_test, y_test_pred, multioutput='raw_values'))
print("MSE Train:", mean_squared_error(y_train, y_train_pred, multioutput='raw_values'))
print("MSE Test:", mean_squared_error(y_test, y_test_pred, multioutput='raw_values'))
print("R2 Train:", r2_score(y_train, y_train_pred, multioutput = 'raw_values'))
print("R2 Test:", r2_score(y_test, y_test_pred, multioutput = 'raw_values'))

MAE Train: [12.62478881 32.89291447  2.34661563  6.10025343]
MAE Test: [33.89938608 87.7542827   6.24656118 16.09829114]
MSE Train: [ 262.54355881 1729.5806264     8.90637408   59.6576679 ]
MSE Test: [ 1939.23188825 12692.24027152    65.8012116    420.06931667]
R2 Train: [0.83848761 0.84162131 0.82183668 0.83300749]
R2 Test: [-0.30513666 -0.17443088 -0.17378971 -0.13521851]


## Option 4: Train a Random Forest on Kernel PCA Reduced Data

(1) Option 4a: Run this option to reduce the data to 200 components

In [9]:
geno_kernel_pca = kernel_pca(genotypes_matrix, 200)

(2) Option 4: Run this option to reduce the data to 50 components

In [16]:
geno_kernel_pca = kernel_pca(genotypes_matrix, 50)

In [17]:
X_geno, y_pheno = get_rats_with_pheno(geno_kernel_pca, loco)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_geno, y_pheno, test_size=0.20)

In [19]:
regr = RandomForestRegressor(n_jobs = -1).fit(X_train, y_train)

In [20]:
y_train_pred = regr.predict(X_train)
y_test_pred = regr.predict(X_test)

In [21]:
print("MAE Train:", mean_absolute_error(y_train, y_train_pred, multioutput='raw_values'))
print("MAE Test:", mean_absolute_error(y_test, y_test_pred, multioutput='raw_values'))
print("MSE Train:", mean_squared_error(y_train, y_train_pred, multioutput='raw_values'))
print("MSE Test:", mean_squared_error(y_test, y_test_pred, multioutput='raw_values'))
print("R2 Train:", r2_score(y_train, y_train_pred, multioutput = 'raw_values'))
print("R2 Test:", r2_score(y_test, y_test_pred, multioutput = 'raw_values'))

MAE Train: [12.00054646 30.82662091  2.12474657  5.50981521]
MAE Test: [31.63093249 80.4614557   5.63162447 15.57622363]
MSE Train: [ 228.16589266 1527.51856389    7.22499319   48.59001283]
MSE Test: [ 1580.78399427 11048.18630443    51.9247403    382.18868038]
R2 Train: [0.85782444 0.85990043 0.85913318 0.86211041]
R2 Test: [-0.00452131 -0.01842162 -0.01731643  0.02131085]


# (B) Fresh Run: Explore how Increasing Regularization Penalty Impacts R2

## Option 1: Run ElasticNet and increase regularization constant with no dimensionality reduction

In [22]:
X_geno, y_pheno = get_rats_with_pheno(genotypes_matrix, loco)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_geno, y_pheno, test_size=0.20)

In [1]:
def run_elastic_net(alpha, max_iter):
    
    regr = ElasticNet(alpha = alpha, max_iter = max_iter).fit(X_train, y_train)
    y_train_pred = regr.predict(X_train)
    y_test_pred = regr.predict(X_test)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    return([alpha, train_r2, test_r2])

In [27]:
alphas = [1,5,10,25,50,100,500,1000,5000,10000]
r2_results = Parallel(n_jobs=6)(delayed(run_elastic_net)(alpha) for alpha in alphas)

In [28]:
r2_results

[[1, 0.4914223822480703, -0.0070173237399553945],
 [5, 0.09575436986325817, 0.018744462857174682],
 [10, 0.02048135437029014, 0.0028678405044191],
 [25, 3.3306690738754696e-16, -0.0030070988515400465],
 [50, 3.3306690738754696e-16, -0.0030070988515400465],
 [100, 3.3306690738754696e-16, -0.0030070988515400465],
 [500, 3.3306690738754696e-16, -0.0030070988515400465],
 [1000, 3.3306690738754696e-16, -0.0030070988515400465],
 [5000, 3.3306690738754696e-16, -0.0030070988515400465],
 [10000, 3.3306690738754696e-16, -0.0030070988515400465]]

In [29]:
alphas = [0.0001, 0.001, 0.01, 0.1, 0.5]
r2_results_2 = Parallel(n_jobs=6)(delayed(run_elastic_net)(alpha) for alpha in alphas)

/Users/zacharywallace/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.682e+04, tolerance: 2.024e+03
  model = cd_fast.enet_coordinate_descent(
/Users/zacharywallace/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.828e+04, tolerance: 3.088e+02
  model = cd_fast.enet_coordinate_descent(
/Users/zacharywallace/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasi

In [30]:
r2_results_2

[[0.0001, 0.9999999232238274, -0.33891468499919797],
 [0.001, 0.9999944966429852, -0.4110740755261058],
 [0.01, 0.9994333925032699, -0.42227117359221955],
 [0.1, 0.9608699845523099, -0.26559118800478687],
 [0.5, 0.6963286803988944, -0.06429047960989878]]

## Option 2: Run ElasticNet and increase regularization constant with PCA reduced data

In [8]:
geno_pca = pca(genotypes_matrix, 250)

In [10]:
X_geno, y_pheno = get_rats_with_pheno(geno_pca, loco)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_geno, y_pheno, test_size=0.20)

In [12]:
regr = ElasticNet(max_iter = 10000).fit(X_train, y_train)

In [13]:
y_train_pred = regr.predict(X_train)
y_test_pred = regr.predict(X_test)

In [14]:
print("MAE Train:", mean_absolute_error(y_train, y_train_pred, multioutput='raw_values'))
print("MAE Test:", mean_absolute_error(y_test, y_test_pred, multioutput='raw_values'))
print("MSE Train:", mean_squared_error(y_train, y_train_pred, multioutput='raw_values'))
print("MSE Test:", mean_squared_error(y_test, y_test_pred, multioutput='raw_values'))
print("R2 Train:", r2_score(y_train, y_train_pred, multioutput = 'raw_values'))
print("R2 Test:", r2_score(y_test, y_test_pred, multioutput = 'raw_values'))

MAE Train: [28.88397036 71.78981506  5.1589985  13.53580593]
MAE Test: [32.76196602 81.94933482  5.85214246 15.29594792]
MSE Train: [1349.18952229 8249.37650833   42.73596514  288.04065404]
MSE Test: [ 1682.97367317 10573.10515355    54.20662814   357.00263254]
R2 Train: [0.16652692 0.25646848 0.16668766 0.20818652]
R2 Test: [-0.10869699 -0.04657238 -0.06139253 -0.03877653]


# (C) Fresh Run: Explore XGBoost Models and Hyperparameter Tuning

### XGBoost with Train and Test

In [8]:
X_geno, y_pheno = get_rats_with_pheno(genotypes_matrix, loco)
X_train, X_test, y_train, y_test = train_test_split(X_geno, y_pheno, test_size=0.20)

In [10]:
xgb = xg.XGBRegressor(objective ='reg:squarederror', n_jobs = -1).fit(X_train, y_train)

In [11]:
y_train_pred = xgb.predict(X_train)
y_test_pred = xgb.predict(X_test)

In [12]:
print("MAE Train:", mean_absolute_error(y_train, y_train_pred, multioutput='raw_values'))
print("MAE Test:", mean_absolute_error(y_test, y_test_pred, multioutput='raw_values'))
print("MSE Train:", mean_squared_error(y_train, y_train_pred, multioutput='raw_values'))
print("MSE Test:", mean_squared_error(y_test, y_test_pred, multioutput='raw_values'))
print("R2 Train:", r2_score(y_train, y_train_pred, multioutput = 'raw_values'))
print("R2 Test:", r2_score(y_test, y_test_pred, multioutput = 'raw_values'))

MAE Train: [0.34063275 0.85358095 0.06780265 0.11579345]
MAE Test: [32.88587865 83.0938272   5.65949354 13.383994  ]
MSE Train: [0.19540265 1.23943147 0.00775575 0.02240661]
MSE Test: [ 1788.76525076 11173.60214298    51.46274191   295.41478995]
R2 Train: [0.99987866 0.99988753 0.99984864 0.99993845]
R2 Test: [-0.15385686 -0.07344891 -0.00963227  0.13658915]


### XGBoost with Hyperparameter Tuning

In [7]:
X_geno, y_pheno = get_rats_with_pheno(genotypes_matrix, loco)
X_train, X_test, y_train, y_test = train_test_split(X_geno, y_pheno, test_size=0.20)

In [8]:
xgb = xg.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.9, 
                      gamma = 5, max_depth = 3, min_child_weight = 5,
                      max_delta_step = 5, subsample = 0.5, alpha = 1, n_jobs = -1).fit(X_train, y_train)

In [9]:
y_train_pred = xgb.predict(X_train)
y_test_pred = xgb.predict(X_test)

In [10]:
print("MAE Train:", mean_absolute_error(y_train, y_train_pred, multioutput='raw_values'))
print("MAE Test:", mean_absolute_error(y_test, y_test_pred, multioutput='raw_values'))
print("MSE Train:", mean_squared_error(y_train, y_train_pred, multioutput='raw_values'))
print("MSE Test:", mean_squared_error(y_test, y_test_pred, multioutput='raw_values'))
print("R2 Train:", r2_score(y_train, y_train_pred, multioutput = 'raw_values'))
print("R2 Test:", r2_score(y_test, y_test_pred, multioutput = 'raw_values'))

MAE Train: [ 12.38717595 181.48627244   2.48416701   6.10341074]
MAE Test: [ 39.58997843 178.29113924  11.18331334  22.13462368]
MSE Train: [2.44697732e+02 4.28240620e+04 1.02102563e+01 6.02859201e+01]
MSE Test: [ 2536.5507155  41311.81962025   213.32248495   775.08009693]
R2 Train: [ 0.84807931 -2.92176957  0.7991296   0.83176127]
R2 Test: [-0.63635216 -2.82553932 -3.05566817 -1.12430351]


### XGBoost with Kernel PCA

In [11]:
geno_kernel_pca = kernel_pca(genotypes_matrix, 200)
X_geno, y_pheno = get_rats_with_pheno(geno_kernel_pca, loco)
X_train, X_test, y_train, y_test = train_test_split(X_geno, y_pheno, test_size=0.20)

In [14]:
xgb = xg.XGBRegressor(objective ='reg:squarederror', gamma = 1, subsample = 0.5, n_jobs = -1).fit(X_train, y_train)

In [15]:
print("MAE Train:", mean_absolute_error(y_train, y_train_pred, multioutput='raw_values'))
print("MAE Test:", mean_absolute_error(y_test, y_test_pred, multioutput='raw_values'))
print("MSE Train:", mean_squared_error(y_train, y_train_pred, multioutput='raw_values'))
print("MSE Test:", mean_squared_error(y_test, y_test_pred, multioutput='raw_values'))
print("R2 Train:", r2_score(y_train, y_train_pred, multioutput = 'raw_values'))
print("R2 Test:", r2_score(y_test, y_test_pred, multioutput = 'raw_values'))

MAE Train: [ 43.19219441 180.70644139   8.16242726  21.70892581]
MAE Test: [ 43.11663924 181.407173    11.37378759  24.3865633 ]
MSE Train: [ 3049.30611324 42398.0905491    104.77899986   760.40848246]
MSE Test: [ 2934.72997901 43013.90822785   203.49648531   940.64477299]
R2 Train: [-0.96927983 -2.91957579 -1.01751955 -1.16337226]
R2 Test: [-0.63329804 -2.83337017 -3.20721569 -1.39190128]


# (D) Model Other Phenotypes

### Chunk 1

In [14]:
chunk1 = pd.read_csv("phenotype_chunks/pheno_chunk1.txt", sep = '\t')
chunk1 = clean_chunks(chunk1)
X_geno, y_pheno = get_rats_with_pheno(genotypes_matrix, chunk1)
print(X_geno.shape)
print(y_pheno.shape)

(2296, 96654)
(2296, 219)


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_geno, y_pheno, test_size=0.20)

In [ ]:
xgb = xg.XGBRegressor(objective ='reg:squarederror', subsample = 0.5, n_jobs = -1).fit(X_train, y_train)